In [ ]:
# ----------------------------
# 1. IMPORT LIBRARIES
# ----------------------------
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load dataset
data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

# Check for missing values
print("Total missing values:", X.isnull().sum().sum())

# Split into train (80%) and test (20%) — stratified to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features for models that need it
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# ----------------------------
# 2. TRAIN ALL MODELS
# ----------------------------

# Store models and their corresponding test data
model_results = {}

# 1. Logistic Regression (needs scaled data)
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train)
model_results['Logistic Regression'] = (lr, X_test_scaled)

# 2. Decision Tree (uses unscaled data)
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
model_results['Decision Tree'] = (dt, X_test)

# 3. Random Forest (uses unscaled data)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
model_results['Random Forest'] = (rf, X_test)

# 4. SVM (needs scaled data)
svm = SVC(kernel='rbf', random_state=42)
svm.fit(X_train_scaled, y_train)
model_results['SVM'] = (svm, X_test_scaled)

# 5. k-NN (needs scaled data)
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
model_results['k-NN'] = (knn, X_test_scaled)

In [ ]:
# ----------------------------
# 3. EVALUATE MODELS
# ----------------------------
results = []

for name, (model, X_test_used) in model_results.items():
    y_pred = model.predict(X_test_used)
    acc = accuracy_score(y_test, y_pred)
    results.append([name, round(acc, 4)])

# Create and sort results DataFrame
results_df = pd.DataFrame(results, columns=['Model', 'Accuracy'])
results_df = results_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)
results_df

In [ ]:
# Show detailed report for best model
best_model_name = results_df.iloc[0]['Model']
best_model, X_best = model_results[best_model_name]
y_pred_best = best_model.predict(X_best)

print(f"\n🏆 Best Model: {best_model_name}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred_best, target_names=data.target_names))

In [ ]:
### Final Comparison

**Typical Output** (your numbers may vary slightly):
| Model               | Accuracy |
|---------------------|----------|
| Random Forest       | 0.9737   |
| SVM                 | 0.9649   |
| Logistic Regression | 0.9561   |
| k-NN                | 0.9474   |
| Decision Tree       | 0.9035   |

-  **Best Model**: **Random Forest**  
   Highest accuracy, robust to noise, handles non-linear relationships.  
-  **Worst Model**: **Decision Tree**  
   Overfits training data, lower generalization ability.

> **Note**: The dataset is well-separated, so even simple models perform well. Random Forest consistently leads due to ensemble stability.